In [2]:
# !pip install faker

In [16]:
import psycopg2

# Database connection details
DB_HOST = "localhost"
DB_NAME = "iot_data"
DB_USER = "postgres"
DB_PASS = "password"
DB_PORT = "5432"

try:
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASS,
        port=DB_PORT
    )
    conn.autocommit = True  # So TRUNCATE runs without needing explicit commit
    cur = conn.cursor()

    # TRUNCATE with CASCADE to handle foreign key dependencies
    cur.execute("""
        TRUNCATE TABLE alerts, sensor_data, devices, sensor_types RESTART IDENTITY CASCADE;
    """)

    print("✅ All tables truncated successfully.")

except Exception as e:
    print(f"❌ Error: {e}")

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()


✅ All tables truncated successfully.


In [17]:
import psycopg2
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

# PostgreSQL connection
conn = psycopg2.connect(
    host="localhost",
    database="iot_data",
    user="postgres",
    password="password"
)
cur = conn.cursor()

# Truncate all tables first
cur.execute("TRUNCATE alerts, sensor_data, devices, sensor_types RESTART IDENTITY CASCADE;")
conn.commit()

# ---------------------
# 1. Insert Sensor Types
# ---------------------
sensor_types = [
    ("Temperature", "°C"),
    ("Humidity", "%"),
    ("Air Quality", "AQI"),
    ("Battery Level", "%"),
]
for stype, unit in sensor_types:
    cur.execute(
        "INSERT INTO sensor_types (sensor_type, unit, changed_on) VALUES (%s, %s, %s)",
        (stype, unit, fake.date_time_between(start_date='-5y', end_date='now'))
    )
conn.commit()

# ---------------------
# 2. Insert Devices
# ---------------------
device_names = ["EnviroMonitor-Alpha", "WeatherSense-Pro", "AirTrack-2000", "PowerGuard-Lite"]
locations = [
    ("Chennai", 13.0827, 80.2707),
    ("Bangalore", 12.9716, 77.5946),
    ("Hyderabad", 17.3850, 78.4867),
    ("Delhi", 28.6139, 77.2090)
]

for name in device_names:
    loc_name, lat, lon = random.choice(locations)
    cur.execute(
        "INSERT INTO devices (device_name, device_type, location, latitude, logitude, registered_at) VALUES (%s, %s, %s, %s, %s, %s)",
        (name, "IoT Sensor Hub", loc_name, str(lat), str(lon),
         fake.date_time_between(start_date='-5y', end_date='now'))
    )
conn.commit()

# ---------------------
# 3. Insert Sensor Data + Alerts
# ---------------------
cur.execute("SELECT device_id, latitude, logitude FROM devices;")
devices = cur.fetchall()

cur.execute("SELECT sensor_id, sensor_type FROM sensor_types;")
sensors = cur.fetchall()

start_date = datetime.now() - timedelta(days=5*365)
end_date = datetime.now()

reading_id_to_time = {}

while start_date <= end_date:
    for device_id, lat, lon in devices:
        for sensor_id, sensor_type in sensors:
            # Generate realistic sensor values
            if sensor_type == "Temperature":
                value = round(random.uniform(15, 45), 2)
                threshold = value > 40 or value < 18
            elif sensor_type == "Humidity":
                value = round(random.uniform(20, 90), 2)
                threshold = value > 85 or value < 25
            elif sensor_type == "Air Quality":
                value = round(random.uniform(50, 250), 2)
                threshold = value > 200
            elif sensor_type == "Battery Level":
                value = round(random.uniform(10, 100), 2)
                threshold = value < 20
            else:
                value = round(random.uniform(0, 100), 2)
                threshold = False

            reading_time = start_date + timedelta(hours=random.randint(0, 23), minutes=random.randint(0, 59))

            # Insert sensor data
            cur.execute(
                "INSERT INTO sensor_data (device_id, sensor_id, sensor_value, reading_time) VALUES (%s, %s, %s, %s) RETURNING reading_id",
                (device_id, sensor_id, value, reading_time)
            )
            reading_id = cur.fetchone()[0]
            reading_id_to_time[reading_id] = reading_time

            # Insert alerts only if threshold exceeded
            if threshold:
                alert_time = reading_time + timedelta(minutes=random.randint(-5, 5))
                if sensor_type == "Temperature":
                    msg = f"Temperature out of safe range: {value}°C"
                elif sensor_type == "Humidity":
                    msg = f"Humidity abnormal: {value}%"
                elif sensor_type == "Air Quality":
                    msg = f"Air Quality index too high: {value} AQI"
                elif sensor_type == "Battery Level":
                    msg = f"Battery critically low: {value}%"
                else:
                    msg = f"Alert: Sensor reading abnormal ({value})"

                cur.execute(
                    "INSERT INTO alerts (device_id, sensor_id, sensor_value, reading_time, alert_type, message) VALUES (%s, %s, %s, %s, %s, %s)",
                    (device_id, sensor_id, value, alert_time, "Threshold Exceeded", msg)
                )

    start_date += timedelta(days=1)

conn.commit()
cur.close()
conn.close()
print("✅ Data generation complete with realistic readings and alerts.")


✅ Data generation complete with realistic readings and alerts.
